In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import model_selection
import matplotlib.pyplot as plt
import torch
import scipy.stats as stats
import torch.nn as nn

In [8]:
X_numpy = np.load('lasso_PCA.npy', allow_pickle=True)
y_numpy = np.load('label.npy', allow_pickle=True)

In [9]:
X = torch.from_numpy(X_numpy.astype(np.float32)).squeeze()
y = torch.from_numpy(y_numpy.astype(np.float32))

In [10]:
#lets split our data
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2,
                                                                           random_state= 51)

In [11]:
#send the data into gpu
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
X_train,X_test, y_train,y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)

In [12]:
# lets create a model for hyperparameter tuning
class NeuralNetworkWithHyper(nn.Module):
    def __init__(self,hidden_layers, num_hidden_units, input_size=8):
        super().__init__()
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.num_hidden_units = num_hidden_units
        
        # Create a list to hold the layers
        self.layers = nn.ModuleList()
        
        #add the input layer
        self.layers.append(nn.Linear(input_size, num_hidden_units))
        self.layers.append(nn.ReLU())
       
        #add hidden layers
        for i in range(hidden_layers - 1):
            self.layers.append(nn.Linear(num_hidden_units, num_hidden_units))
            self.layers.append(nn.ReLU())
        #add the output layer
        self.layers.append(nn.Linear(num_hidden_units, 1))
        
        #add forward function
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [15]:
#Lets set our loss function and optimizer
import optuna

def objective(trial):
    
    #define the search space
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-3)
    hidden_layers = trial.suggest_int("hidden_layers", 1, 1)
    num_hidden_units = trial.suggest_int("num_hidden_units", 26, 46)
    epoch_count = trial.suggest_int("epoch_count", 9000, 12000)
    #weight_decay = trial.suggest_float("weight_decay", 1e-4, 1e-3)
    
    #create the model
    model = NeuralNetworkWithHyper(hidden_layers, 
                                   num_hidden_units 
                                   ).to('mps')
    optimizer = torch.optim.SGD(model.parameters(), learning_rate, #weight_decay
                               )
    loss_fn = torch.nn.L1Loss()
    
    #testing loop
    for epoch in range(epoch_count):
        #convert into train
        model.train()

        #fit data
        train_pred = model(X_train)

        #calculate loss
        train_loss = loss_fn(train_pred, y_train)


        #zero gradients
        optimizer.zero_grad()

        #back propagation
        train_loss.backward()

        #gradient descent
        optimizer.step()

        #testing
        model.eval()
        with torch.inference_mode():
            test_pred = model(X_test)
            test_loss = loss_fn(test_pred, y_test)
            spear_pred = test_pred.to('cpu')
            
    #spearman correlation coefficient       
    validation_metric = stats.spearmanr(y_test.to('cpu'), spear_pred).statistic
    
    #save model weights
    trial.set_user_attr('model_weights', model.state_dict())
    
    return validation_metric

#Create an Optuna Study
study = optuna.create_study(direction='maximize')

#Perform hyperparameter optimization
study.optimize(objective, n_trials=100)

#Print best spearman correlation coefficient
best_validation_metric = study.best_value

[I 2023-11-12 02:14:36,351] A new study created in memory with name: no-name-6911dd41-b753-4595-a07b-bd1a163560a8
[I 2023-11-12 02:14:54,971] Trial 0 finished with value: 0.2329296147964622 and parameters: {'learning_rate': 0.0009777250939934864, 'hidden_layers': 1, 'num_hidden_units': 37, 'epoch_count': 9442}. Best is trial 0 with value: 0.2329296147964622.
[I 2023-11-12 02:15:12,753] Trial 1 finished with value: 0.09463255911726687 and parameters: {'learning_rate': 0.00012391858859659458, 'hidden_layers': 1, 'num_hidden_units': 28, 'epoch_count': 9005}. Best is trial 0 with value: 0.2329296147964622.
[I 2023-11-12 02:15:36,067] Trial 2 finished with value: 0.21709736561768134 and parameters: {'learning_rate': 0.0009976479019921482, 'hidden_layers': 1, 'num_hidden_units': 46, 'epoch_count': 11656}. Best is trial 0 with value: 0.2329296147964622.
[I 2023-11-12 02:15:59,800] Trial 3 finished with value: 0.2205644158461591 and parameters: {'learning_rate': 0.000862842611859144, 'hidden_l

In [14]:
best_trial = study.best_trial
best_params = best_trial.params
best_score = best_trial.value
print(f'Best Score: {best_score} \nHyperparameters: ')
for key,value in best_params.items():
    print(f'{key}: {value}')

Best Score: 0.28971642285696886 
Hyperparameters: 
learning_rate: 0.0009605640784529591
hidden_layers: 1
num_hidden_units: 26
epoch_count: 9984
weight_decay: 0.00042473963315697486
